Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.
Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.


In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('water.txt', delimiter='\t')

In [5]:
data.head(10)

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18
5,North,Bolton,1558,10
6,North,Bootle,1807,15
7,South,Bournemouth,1299,78
8,North,Bradford,1637,10
9,South,Brighton,1359,84


In [8]:
data.corr()

,mortality,hardness
mortality,1.000000,-0.654849
hardness,-0.654849,1.000000


In [9]:
# spearman
data.corr(method ='spearman')

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


In [10]:
# разделим на south и north
data_south = data[data['location'] == 'South']
data_north = data[data['location'] == 'North']

In [11]:
data_south.head(10)

,location,town,mortality,hardness
0,South,Bath,1247,105
2,South,Birmingham,1466,5
7,South,Bournemouth,1299,78
9,South,Brighton,1359,84
10,South,Bristol,1392,73
12,South,Cardiff,1519,21
13,South,Coventry,1307,78
14,South,Croydon,1254,96
18,South,East Ham,1318,122
19,South,Exeter,1260,21


In [12]:
data_north.head(10)

,location,town,mortality,hardness
1,North,Birkenhead,1668,17
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18
5,North,Bolton,1558,10
6,North,Bootle,1807,15
8,North,Bradford,1637,10
11,North,Burnley,1755,12
15,North,Darlington,1491,20
16,North,Derby,1555,39
17,North,Doncaster,1428,39


In [13]:
# считаем корреляцию
data_north.corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


In [14]:
data_south.corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [17]:
import numpy as np

In [50]:
# составим матрицу сопряженности
data_bar = np.array([[203.,239.], [718.,515.]])

In [ ]:
print(data_bar)

In [39]:
print(data_bar[0][1])

515


In [51]:
def matthew_corr(a,b,c,d):
    return (a * d - c * b) / np.sqrt( (a + b) * (a + c) * (d + b) * (d + c) )

In [52]:
print(*data_bar.flatten())

203.0 239.0 718.0 515.0


In [53]:
print(matthew_corr(*data_bar.flatten()))

-0.10900237458678963


В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры

In [55]:
from scipy import stats

In [56]:
print(stats.chi2_contingency(data_bar))

(19.40753078854304, 1.0558987006638725e-05, 1, array([[243.03402985, 198.96597015],
       [677.96597015, 555.03402985]]))


In [64]:
np.sum(data_bar ,axis = 0)

array([921., 754.])

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.



In [65]:
def propotional_conf_int(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)

    p1 = sample1[0] / np.sum(sample1)
    p2 = sample2[0] / np.sum(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ np.sum(sample1) + p2 * (1 - p2)/ np.sum(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ np.sum(sample1) + p2 * (1 - p2)/ np.sum(sample2))
    
    return (left_boundary, right_boundary)

In [66]:
print(propotional_conf_int(data_bar[:,1], data_bar[:,0]))

(0.053905233215813156, 0.13922183141523897)


Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры

In [67]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = np.sum(sample1)
    n2 = np.sum(sample2)
    
    p1 = sample1[0] / n1
    p2 = sample2[0] / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [70]:
def propotional_test_int(z, alternative = 'two-sided'):
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(abs(z)))
    elif alternative == 'greater':
        return 1 - stats.norm.cdf(z)
    elif alternative == 'less':
        return stas.norm.cdf(z)

In [71]:
print(propotional_test_int( proportions_diff_z_stat_ind(data_bar[:,0], data_bar[:, 1]) )  )

8.153453089576601e-06


Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

||Не доволен| Более или менее | Доволен |
| --- | --- | --- | --- |
| Не очень счастлив| 197 | 111 | 33 |
| Достаточно счастлив| 382 | 685 | 331|
| Очень счастлив | 110 | 342 | 333 |

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.



In [74]:
general_social = np.array([[197., 111., 33.],
                         [382., 685., 331.],
                         [110., 342., 333.]])

In [75]:
stats.chi2_contingency(general_social)

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

In [76]:
def find_v_kramer(matrix):
    K = 3
    n = matrix.sum()
    return np.sqrt(scipy.stats.chi2_contingency(matrix)[0] / (n * (K - 1) ))

In [77]:
print(find_v_kramer(general_social))

0.2412013934500338
